In [1]:
import pandas as pd

In [5]:
df = pd.read_csv("spam_ham_dataset.csv")
df.head()

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\nth...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\n( see a...",0
2,3624,ham,"Subject: neon retreat\nho ho ho , we ' re arou...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\nthis deal is to ...,0


In [9]:
df = df[ ["text","label_num"] ] 
df.head(1)

,text,label_num
0,Subject: enron methanol ; meter # : 988291\nth...,0


In [11]:
df.iloc[0,0]

"Subject: enron methanol ; meter # : 988291\nthis is a follow up to the note i gave you on monday , 4 / 3 / 00 { preliminary\nflow data provided by daren } .\nplease override pop ' s daily volume { presently zero } to reflect daily\nactivity you can obtain from gas control .\nthis change is needed asap for economics purposes ."

In [15]:
!pip install nltk
# Native Language Tool Kit

In [39]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mete\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [49]:
# Python , PYThon , PYTHOn kelimelerinin hepsinin aynı kelime olduğunu anlatma
# Python. , Python : gibi kelimelerin hepsinin de aynı olduğunu anlatma, noktalamaları kaldırma
import string
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
text = df.iloc[0,0]
text = text.lower()
text = text.translate(str.maketrans('','', string.punctuation))
words = text.split() # Merhaba, Python 3 -> ["Merhaba"] , ["Python 3"] # [Merhaba,Python,3]
words = [word for word in words if not word.isdigit()]
# Stem -> Kelimeleri köküne indirir (Koşuyor -> Koş), Remove Stopwords
# Remove Stopwords -> "The,at,is" gibi kelimeleri kaldırır ve daha net bir anlam ortaya çıkar
words = [ ps.stem(word) for word in words if word not in stopwords.words("english")]
" ".join(words)

'subject enron methanol meter follow note gave monday preliminari flow data provid daren pleas overrid pop daili volum present zero reflect daili activ obtain ga control chang need asap econom purpos'

In [59]:
def clean_text(text) : 
    text = text.lower()
    text = text.translate(str.maketrans('','', string.punctuation))
    words = text.split() # Merhaba, Python 3 -> ["Merhaba"] , ["Python 3"] # [Merhaba,Python,3]
    words = [word for word in words if not word.isdigit()]
    # Stem -> Kelimeleri köküne indirir (Koşuyor -> Koş), Remove Stopwords
    # Remove Stopwords -> "The,at,is" gibi kelimeleri kaldırır ve daha net bir anlam ortaya çıkar
    words = [ ps.stem(word) for word in words if word not in stopwords.words("english")]
    return " ".join(words)

In [63]:
clean_text("Merhaba, bu e-posta 45. Python dersi hakkındadır.")

'merhaba bu eposta python dersi hakkındadır'

In [65]:
df['clean_text'] = df['text'].apply(clean_text)

In [66]:
df.head()

,text,label_num,clean_text
0,Subject: enron methanol ; meter # : 988291\nth...,0,subject enron methanol meter follow note gave ...
1,"Subject: hpl nom for january 9 , 2001\n( see a...",0,subject hpl nom januari see attach file hplnol...
2,"Subject: neon retreat\nho ho ho , we ' re arou...",0,subject neon retreat ho ho ho around wonder ti...
3,"Subject: photoshop , windows , office . cheap ...",1,subject photoshop window offic cheap main tren...
4,Subject: re : indian springs\nthis deal is to ...,0,subject indian spring deal book teco pvr reven...


In [69]:
x = df["clean_text"]
y = df["label_num"]

# Vectorize -> IfIDF Vectorizer 
# Vectorize nedir? ÖDEV
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(df["clean_text"])
X_tfidf

<5171x37932 sparse matrix of type '<class 'numpy.float64'>'
	with 307420 stored elements in Compressed Sparse Row format>

In [71]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_tfidf , y , test_size=0.3 , random_state=42)

In [77]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(X_train,y_train)

MultinomialNB()

In [79]:
y_pred = model.predict(X_test)

In [81]:
y_pred

array([0, 1, 0, ..., 0, 0, 0], dtype=int64)

In [83]:
from sklearn.metrics import classification_report

In [85]:
print(classification_report(y_test, y_pred))

# Support -> Veri Sayısı
# Precision -> Kesinlik -> Söylediğim ne kadar doğruydu?
# Recall -> Duyarlılık -> Ne kadarını yakalayabildim? -> Spamlarda yüzde 32'lik bir sızma var
# F1-Score -> Precision/Recall Oranı

              precision    recall  f1-score   support

           0       0.89      1.00      0.94      1121
           1       1.00      0.68      0.81       431

    accuracy                           0.91      1552
   macro avg       0.95      0.84      0.88      1552
weighted avg       0.92      0.91      0.91      1552



In [ ]:
import pandas as pd
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

nltk.download("stopwords")

In [ ]:
df = pd.read_csv("spam_ham_dataset.csv")
df = df[["text", "label_num"]]

In [ ]:
ps = PorterStemmer()
stop_words = set(stopwords.words("english"))

def clean_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    words = text.split()
    words = [ps.stem(word) for word in words if word not in stop_words and not word.isdigit()]
    return " ".join(words)

df["clean_text"] = df["text"].apply(clean_text)

In [ ]:
tfidf = TfidfVectorizer(max_features=3000, ngram_range=(1, 2))
X = tfidf.fit_transform(df["clean_text"]).toarray()
y = df["label_num"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))